In [39]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, KFold

In [40]:
df = pd.read_csv('data/train_engineered.csv')
df

,Delivery_person_Age,Delivery_person_Ratings,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Order_Pickup_Time,Delivery_Distance (m),Ordered_month,Ordered_year,Ordered_time (mins)
0,37.0,4.9,Sunny,High,2.0,Snack,motorcycle,0.0,No,Urban,24,15.0,3025.149286,3,2022,690
1,34.0,4.5,Stormy,Jam,2.0,Snack,scooter,1.0,No,Metropolitian,33,5.0,20183.529729,3,2022,1185
2,23.0,4.4,Sandstorms,Low,0.0,Drinks,motorcycle,1.0,No,Urban,26,15.0,1552.757802,3,2022,510
3,38.0,4.7,Sunny,Medium,0.0,Buffet,motorcycle,1.0,No,Metropolitian,21,10.0,7790.401360,4,2022,1080
4,32.0,4.6,Cloudy,High,1.0,Snack,scooter,1.0,No,Metropolitian,30,15.0,6210.137966,3,2022,810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41363,30.0,4.8,Windy,High,1.0,Meal,motorcycle,0.0,No,Metropolitian,32,10.0,1489.845884,3,2022,695
41364,21.0,4.6,Windy,Jam,0.0,Buffet,motorcycle,1.0,No,Metropolitian,36,15.0,11007.734764,2,2022,1195
41365,30.0,4.9,Cloudy,Low,1.0,Drinks,scooter,0.0,No,Metropolitian,16,15.0,4657.195394,3,2022,1430
41366,20.0,4.7,Cloudy,High,0.0,Snack,motorcycle,1.0,No,Metropolitian,26,5.0,6232.393050,3,2022,815


# Train/Valid/Test Split

In [41]:
from sklearn.model_selection import train_test_split, cross_val_score

In [42]:
X = df.drop('Time_taken(min)', axis=1)
Y = df['Time_taken(min)']

xtrain, xtest, ytrain, ytest = train_test_split(X,Y, test_size= 0.25)

# Linear Classifiers
- Lasso
- Ridge
- Polynomial

In [43]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

## Polynomial

In [44]:
linreg = make_pipeline(FunctionTransformer(pd.get_dummies, kw_args={'drop_first':True}), LinearRegression()).fit(xtrain, ytrain)

In [45]:
mean_squared_error(linreg.predict(xtest), ytest)

37.15453154836071

## Lasso

In [46]:
alpha = [0.1, 0.3, 0.5, 0.75, 1, 3, 5, 10]
ridgecv = make_pipeline(FunctionTransformer(pd.get_dummies, kw_args={'drop_first':True}), RidgeCV(alphas=alpha, cv=5)).fit(xtrain,ytrain)

/Users/ehniv/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.46807e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/ehniv/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.32268e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/ehniv/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.18803e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/ehniv/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.52658e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/ehniv/opt/anaconda3/lib/pytho

In [47]:
# get cross validation results for ridge regression
mean_squared_error(ridgecv.predict(xtest), ytest)

37.15452078052192

## Lasso Regression

In [48]:
alpha = [0.1, 0.3, 0.5, 0.75, 1, 3, 5, 10]
lassocv = make_pipeline(FunctionTransformer(pd.get_dummies, kw_args={'drop_first':True}), LassoCV(alphas=alpha, cv=5)).fit(xtrain,ytrain)

In [49]:
# get cross validation results for lasso regression
mean_squared_error(lassocv.predict(xtest), ytest)

39.04620848476174

# Random Forest

In [50]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [51]:
# Grid Search
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 320, stop = 380, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(30, 50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [4, 5, 6]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 3, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

rf_model = make_pipeline(FunctionTransformer(pd.get_dummies, kw_args={'drop_first':True}), RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)).fit(xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
# hyperparameter tuning
rf_model['randomizedsearchcv'].best_params_

In [ ]:
mean_squared_error(ytest, rf_model.predict(xtest))

### Feature Importance

In [ ]:
start_time = time.time()
rf_model_tree = rf_model['randomizedsearchcv'].feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_model['randomizedsearchcv'].estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

In [ ]:
forest_importance = pd.Series(rf_model_tree, index=df.columns.values)

fig, ax = plt.subplots()
forest_importance.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

# KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import Normalizer

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
knn_hyper = {'model__n_neighbors':[21,22,23,24],
              'model__weights':['uniform','distance']}

knn = Pipeline([('onehot', FunctionTransformer(pd.get_dummies, kw_args={'drop_first':True})),
                           ('normalize', Normalizer()),
                           ('model', KNeighborsRegressor())
                           ])

knn_grid = GridSearchCV(estimator=knn, param_grid=knn_hyper, cv=kfold, scoring='neg_mean_squared_error')
knn_grid.fit(xtrain,ytrain)

In [ ]:
knn_grid.best_estimator_

In [ ]:
mean_squared_error(ytest, knn_grid.predict(xtest))